# Detección de Fraude en Tarjetas de Crédito: Aprovechando el Poder de **Machine Learning** en **Snowflake ML**

El fraude con tarjetas de crédito es una preocupación significativa en la era digital, donde se realizan miles de millones de transacciones diarias en todo el mundo. Detectar actividades fraudulentas de manera rápida y precisa es fundamental para proteger tanto a los consumidores como a las instituciones financieras. **Machine Learning** (**ML**) ha emergido como una herramienta poderosa en la lucha contra el fraude con tarjetas de crédito, ofreciendo métodos sofisticados para identificar actividades sospechosas en tiempo real.

## El Desafío de la Detección de Fraude
El fraude con tarjetas de crédito puede tomar diversas formas, incluyendo transacciones no autorizadas, toma de control de cuentas y fraude con identidades sintéticas. Los sistemas tradicionales basados en reglas, que dependen de patrones y umbrales predefinidos, a menudo no son efectivos para detectar esquemas de fraude nuevos y en evolución. Estos sistemas también pueden generar un alto número de **false positives** (falsos positivos), lo que lleva a rechazos innecesarios de transacciones y a la insatisfacción de los clientes





### **Importar Librerías**

Para comenzar, haz clic en el botón **Start** (Iniciar). Una vez que diga **Active** (Activo), estarás listo para ejecutar el resto del Notebook. Todos los paquetes ya han sido pre-cargados. En la celda siguiente, importa los paquetes de Python necesarios y configura una sesión de **Snowflake**.

### **Snowflake ML Feature Store**  
Un **Python SDK** para definir, registrar, recuperar y gestionar **features**.

**Entity** (Entidad): Las entidades son los objetos subyacentes con los que las **features** y **feature views** están asociadas. Encapsulan las claves de unión utilizadas para buscar las **features**.

**FeatureView** (Vista de Feature): Una **feature view** es un grupo de **features** lógicamente relacionadas que se actual



In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark import Window
from snowflake.snowpark.functions import *

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"credit_card_fraud", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":0, "source":"notebook"}}


from snowflake.snowpark import functions as F
from snowflake.ml.feature_store import (
FeatureStore,
FeatureView,
Entity,
CreationMode
)
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions


Configurar los objetos de base de datos y los warehouses necesarios

In [ ]:
session.sql("USE ROLE SYSADMIN").collect()
session.sql("CREATE DATABASE IF NOT EXISTS CC_FINS_DB").collect()
session.sql("USE database CC_FINS_DB").collect()

session.sql("CREATE SCHEMA IF NOT EXISTS ANALYTICS").collect()
session.sql("CREATE WAREHOUSE if not exists CC_FINS_WH").collect()


### Configurar variables

Esto se hace principalmente para crear un rol separado para un consumidor y productor de **Feature Store**. Para simplificar, consideraremos el mismo rol, **SYSADMIN**, en este caso, tanto para el productor como para el consumidor.


In [ ]:
USE ROLE ACCOUNTADMIN;
SET FS_ROLE_PRODUCER = 'SYSADMIN';
SET FS_ROLE_CONSUMER = 'SYSADMIN';
SET FS_DATABASE = 'CC_FINS_DB';
SET FS_SCHEMA = 'ANALYTICS';
SET SCHEMA_FQN = CONCAT($FS_DATABASE, '.', $FS_SCHEMA);
SET FS_WAREHOUSE = 'CC_FINS_WH';


-- Create roles

CREATE SCHEMA IF NOT EXISTS IDENTIFIER($FS_SCHEMA);



Crea un nuevo **Feature Store** especificando la base de datos y el esquema donde se almacenarán las **features**. Ten en cuenta que también configuramos un **default_warehouse** que se utilizará con el **Feature Store**.


In [ ]:
session.sql('USE ROLE SYSADMIN').collect()
session.sql('USE WAREHOUSE CC_FINS_WH').collect()
session.sql('USE SCHEMA ANALYTICS').collect()

FS1=FeatureStore(
session=session,
database="CC_FINS_DB",
    name="ANALYTICS",
    default_warehouse="CC_FINS_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST)

In [ ]:
session.sql("USE ROLE SYSADMIN").collect()
session.sql("USE DATABASE CC_FINS_DB").collect()
session.sql("USE SCHEMA ANALYTICS").collect()

Imprimir el esquema

In [ ]:
# Current Environment Details
print('Connection Established with the following parameters:')
print('User      : {}'.format(session.get_current_user()))
print('Role      : {}'.format(session.get_current_role()))
print('Database  : {}'.format(session.get_current_database()))
print('Schema    : {}'.format(session.get_current_schema()))
print('Warehouse : {}'.format(session.get_current_warehouse()))

### La siguiente celda realiza varias acciones:

1. Crea un formato de archivo denominado **CSVFORMAT**.
2. Crea un **stage** denominado **CREDITCARD_TRANSACTIONS**.
3. El **stage** apunta al almacenamiento externo donde se encuentra el conjunto de datos de transacciones de tarjetas de crédito.
4. Lista el contenido del **stage** **CREDITCARD_TRANSACTIONS**.


In [ ]:
-- Create csv format

CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1
    TYPE = 'CSV';

-- Create a stage with the csv format to stage the credit card transactions
CREATE STAGE IF NOT EXISTS CREDITCARD_TRANSACTIONS
    FILE_FORMAT =  CSVFORMAT 
    URL = 's3://sfquickstarts/sfguide_credit_card_fraud_detection_using_snowflakeml/fraud_detection_dataset.csv';
    
    
--Upload the file manually in case of internal stage or in case of external stage point to file storage
-- Inspect content of stage
LS @CREDITCARD_TRANSACTIONS;


Leer el archivo CSV desde el **stage** de Snowflake, crear un **DataFrame** a partir de él y luego escribir este **DataFrame** en una tabla de Snowflake.


In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
transactions_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@CREDITCARD_TRANSACTIONS")

transactions_df.write.mode('overwrite').save_as_table('CREDITCARD_TRANSACTIONS')

transactions_df.show()


In [ ]:
update CREDITCARD_TRANSACTIONS
set merchant = case 
                    when merchant= 'newegg.com' then 'mercadolibre.com'
                    when merchant= 'homedepot.com' then 'homecenter.com'
                    when merchant= 'etsy.com' then 'temu.com'
                    when merchant= 'lowes.com' then 'airbnb.com'
                    when merchant= 'walmart.com' then 'olimpica.com'
                    when merchant= 'bestbuy.com' then 'falabella.com'
                    when merchant= 'globalexp.com' then 'exito.com'
                    else merchant
                end;

update CREDITCARD_TRANSACTIONS
set location = case 
                    when location= 'Moscow' then 'Bogota'
                    when location= 'Atlanta' then 'Medellin'
                    when location= 'Kansas City' then 'Cali'
                    when location= 'Los Angeles' then 'Cartagena'
                    when location= 'Tucson' then 'Barranquilla'
                    when location= 'Chicago' then 'Santa Marta'
                    when location= 'Detroit' then 'Neiva'
                    when location= 'Las Vegas' then 'Villavicencio'
                    when location= 'Fresno' then 'Ibague'
                    when location= 'Washington DC' then 'Yopal'
                    when location= 'Miami' then 'San Andres'
                    when location= 'Oklahoma City' then 'Pasto'
                    when location= 'Albuquerque' then 'Manizales'
                    when location= 'Beijing' then 'Pereira'
                    when location= 'Sacramento' then 'Tunja'
                    when location= 'Seattle' then 'Bucaramanga'
                    when location= 'San Francisco' then 'Popayan'
                    when location= 'Boston' then 'Valledupar'
                    when location= 'San Diego' then 'Rioacha'
                    when location= 'El Paso' then 'Cucuta'
                    when location= 'Nashville' then 'Armenia'
                    when location= 'Omaha' then 'Sincelejo'
                    when location= 'Jacksonville' then 'Envigado'
                    when location= 'Denver' then 'Bello'
                    when location= 'Milwaukee' then 'Chia'
                    when location= 'Raleigh' then 'Cota'
                    when location= 'Virginia Beach' then 'Acacias'
                    when location= 'San Antonio' then 'Florencia'
                    when location= 'Houston' then 'Arauca'
                    when location= 'Phoenix' then 'Leticia'
                    when location= 'Indianapolis' then 'Doradal'
                    when location= 'Austin' then 'Soledad'
                    when location= 'Columbus' then 'Duitama'
                    when location= 'Portland' then 'Pto Carreno'
                    when location= 'Dallas' then 'Quibdo'
                    when location= 'Philadelphia' then 'Mocoa'
                    when location= 'New York' then 'Jamundi'
                    when location= 'Mesa' then 'Buenaventura'
                    else location
                end;


update CREDITCARD_TRANSACTIONS
set latitude = case
                    when location= 'Bogota' then 4.7110
                    when location = 'Medellin' then 6.2442
                    when location = 'Cali' then 3.4516
                    when location = 'Cartagena' then 10.3910
                    when location = 'Barranquilla' then 10.9630
                    when location = 'Santa Marta' then 11.2408
                    when location = 'Neiva' then 2.9386
                    when location = 'Villavicencio' then 4.1403
                    when location = 'Ibague' then 4.4381
                    when location = 'Yopal' then 5.3343
                    when location = 'San Andres' then 12.5848
                    when location = 'Pasto' then 1.2135
                    when location = 'Manizales' then 5.0689
                    when location = 'Pereira' then 4.8130
                    when location = 'Tunja' then 5.5346
                    when location = 'Bucaramanga' then 7.1195
                    when location = 'Popayan' then 2.4379
                    when location = 'Valledupar' then 10.4637
                    when location = 'Rioacha' then 11.5375
                    when location = 'Cucuta' then 7.8930
                    when location = 'Armenia' then 4.5339
                    when location = 'Sincelejo' then 9.3136
                    when location = 'Envigado' then 6.1710
                    when location = 'Bello' then 6.3583
                    when location = 'Chia' then 4.8786
                    when location = 'Cota' then 4.7372
                    when location = 'Acacias' then 3.9886
                    when location = 'Florencia' then 1.6115
                    when location = 'Arauca' then 7.0873
                    when location = 'Leticia' then -4.2045
                    when location = 'Doradal' then 6.5297
                    when location = 'Soledad' then 10.8393
                    when location = 'Duitama' then 5.8564
                    when location = 'Pto Carreno' then 6.1880
                    when location = 'Quibdo' then 5.6947
                    when location = 'Mocoa' then 1.1536
                    when location = 'Jamundi' then 3.3069
                    when location = 'Buenaventura' then 3.8837
                    else latitude
                end;

update CREDITCARD_TRANSACTIONS
set             
longitude = case
                when location = 'Bopgota' then -74.0721
                when location = 'Medellin' then -75.5736
                when location = 'Cali' then -76.5320
                when location = 'Cartagena' then -75.4792
                when location = 'Barranquilla' then -74.7967
                when location = 'Santa Marta' then -74.2004
                when location = 'Neiva' then -75.2803
                when location = 'Villavicencio' then -73.6328
                when location = 'Ibague' then -75.2000
                when location = 'Yopal' then -72.3921
                when location = 'San Andres' then -81.7111
                when location = 'Pasto' then -77.2811
                when location = 'Manizales' then -75.5183
                when location = 'Pereira' then -75.6962
                when location = 'Tunja' then -73.3674
                when location = 'Bucaramanga' then -73.1198
                when location = 'Popayan' then -77.2805
                when location = 'Valledupar' then -73.2484
                when location = 'Rioacha' then -71.9191
                when location = 'Cucuta' then -72.5078
                when location = 'Armenia' then -75.6817
                when location = 'Sincelejo' then -75.3952
                when location = 'Envigado' then -75.5805
                when location = 'Bello' then -75.5637
                when location = 'Chia' then -74.0312
                when location = 'Cota' then -74.2359
                when location = 'Acacias' then -73.8881
                when location = 'Florencia' then -75.6111
                when location = 'Arauca' then -70.7600
                when location = 'Leticia' then -69.9437
                when location = 'Doradal' then -74.5773
                when location = 'Soledad' then -74.7942
                when location = 'Duitama' then -73.0202
                when location = 'Pto Carreno' then -67.3185
                when location = 'Quibdo' then -76.6473
                when location = 'Mocoa' then -76.6489
                when location = 'Jamundi' then -76.6394
                when location = 'Buenaventura' then -77.0792
                else longitude
                end;

Generar diversas **features** relacionadas con los patrones de gasto del usuario

Estas **features** incluyen el gasto semanal, mensual y anual, así como estadísticas transaccionales que se pueden utilizar para análisis posteriores o tareas de **machine learning**, como la detección de fraude.


In [ ]:
WITH 
weekly_spending AS (
    SELECT
        USER_ID,
        DATE_TRUNC('week',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')) AS week,
        SUM(TRANSACTION_AMOUNT) AS total_spent_wk
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID,
        DATE_TRUNC('week',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI'))
),
mean_weekly_spending AS (
    SELECT
        USER_ID,
        AVG(total_spent_wk) AS mean_weekly_spent
    FROM
        weekly_spending
    GROUP BY
        USER_ID
),
monthly_spending AS (
    SELECT
        USER_ID,
        DATE_TRUNC('month',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')) AS month,
        SUM(TRANSACTION_AMOUNT) AS total_spent
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID,
        DATE_TRUNC('month',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI'))
),
mean_monthly_spending AS (
    SELECT
        USER_ID,
        AVG(total_spent) AS mean_monthly_spent
    FROM
        monthly_spending
    GROUP BY
        USER_ID
),
yearly_spending AS (
    SELECT
        USER_ID,
        DATE_TRUNC('year',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')) AS month,
        SUM(TRANSACTION_AMOUNT) AS total_spent_yr
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID,
        DATE_TRUNC('year',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI'))
),
mean_yearly_spending AS (
    SELECT
        USER_ID,
        AVG(total_spent_yr) AS mean_yearly_spent
    FROM
        yearly_spending
    GROUP BY
        USER_ID
),
features AS (
    SELECT
        USER_ID,
        COUNT(TRANSACTION_ID) AS total_transactions,
        AVG(TRANSACTION_AMOUNT) AS avg_per_transaction_amount,
        STDDEV(TRANSACTION_AMOUNT) AS stddev_transaction_amount,
        COUNT(DISTINCT MERCHANT) AS num_unique_merchants,
        round(COUNT(TRANSACTION_ID) / COUNT(DISTINCT DATE_TRUNC('month',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')),0)) AS transactions_per_month,
        COUNT(CASE WHEN IS_FRAUD = 1 THEN 1 END) * 1.0 / COUNT(TRANSACTION_ID) AS fraud_rate
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID
)
SELECT
    DISTINCT cc.USER_ID,total_transactions,avg_per_transaction_amount,stddev_transaction_amount,num_unique_merchants,
    ROUND(mws.mean_weekly_spent,2) as mean_weekly_spent,
    ROUND(mms.mean_monthly_spent,2) as mean_monthly_spent,
    ROUND(mys.mean_yearly_spent,2) as mean_yearly_spent
FROM
    CREDITCARD_TRANSACTIONS cc
    JOIN features fs ON cc.USER_ID = fs.User_ID
    JOIN mean_monthly_spending mms ON cc.USER_ID = mms.User_ID
    JOIN mean_yearly_spending mys ON cc.USER_ID = mys.User_ID
    JOIN mean_weekly_spending mws ON cc.USER_ID = mws.User_ID;


Generar un **DataFrame** de **features** de clientes y guardarlo como una tabla en Snowflake


In [ ]:
cust_spdf=Generate_Customer_Features.to_df()

cust_spdf.write.mode('overwrite').save_as_table('customer_features')
cust_sdf=session.sql("select * from customer_features")


In [ ]:
cust_sdf.show()

Generar métricas comportamentales acumulativas para los usuarios basadas en sus datos de transacciones, como clics acumulativos y logins acumulativos por hora.

Esto implica el uso de **window functions** (funciones de ventana) y **joins** para combinar y transformar los datos de la tabla **CREDITCARD_TRANSACTIONS**.


In [ ]:
WITH cumulative_behavior AS (
    SELECT
        USER_ID,
        SESSION_ID,
        TRANSACTION_DATE,
        SUM(CLICKS) OVER (PARTITION BY USER_ID ORDER BY TRANSACTION_DATE RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_clicks,
        SUM(LOGIN_PER_HOUR) OVER (PARTITION BY USER_ID ORDER BY TRANSACTION_DATE RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_logins_per_hour
    FROM CREDITCARD_TRANSACTIONS
),
unique_transactions AS (
    SELECT DISTINCT
        USER_ID,
        SESSION_ID,
        TRANSACTION_DATE,
        TRANSACTION_ID,
        TIME_ELAPSED,
        CLICKS,
        LOCATION,
        LATITUDE,
        LONGITUDE
    FROM CREDITCARD_TRANSACTIONS
)
SELECT
    ut.SESSION_ID,
    ut.TRANSACTION_DATE,
    ut.TIME_ELAPSED,
    ut.CLICKS,
    ut.TRANSACTION_ID,
    ut.LOCATION,
    ut.LATITUDE,
    ut.LONGITUDE,
    cb.cumulative_clicks,
    cb.cumulative_logins_per_hour
FROM
    unique_transactions ut
    JOIN cumulative_behavior cb ON ut.USER_ID = cb.USER_ID 
    AND ut.SESSION_ID = cb.SESSION_ID
    AND ut.TRANSACTION_DATE = cb.TRANSACTION_DATE
ORDER BY
    ut.TRANSACTION_ID;


Generar un **DataFrame** de **features** de transacciones y guardarlo como una tabla en Snowflake


In [ ]:
trans_df=Generate_Transaction_Features.to_df()

trans_df.write.mode('overwrite').save_as_table('transaction_features')
trans_sdf=session.sql("select * from transaction_features")

In [ ]:
trans_df.show()

Crear Entities

Una **entity** (entidad) es una abstracción sobre un conjunto de claves primarias utilizadas para buscar datos de **features**. Una **entity** representa un "objeto" del mundo real que tiene datos asociados. La celda a continuación registra una **entity** para **Customer** (Cliente) y **Transaction** (Transacción) en el **Feature Store**.


In [ ]:

# Snowflake Feature Store requires an "entity" with "join_keys" be registered
customer_entity = Entity(name="UserId", join_keys=["USER_ID"])
FS1.register_entity(customer_entity)

transaction_entity = Entity(name="transId", join_keys=["TRANSACTION_ID"])
FS1.register_entity(transaction_entity)


 Listar las **Feature entities**

In [ ]:
FS1.list_entities().show(15)

# Usando **Feature Views**

Un **Feature View** es un grupo de **features** lógicamente relacionadas que se actualizan en el mismo horario. El constructor de **FeatureView** acepta un **Snowpark DataFrame** que contiene la lógica de generación de **features**. El **DataFrame** proporcionado debe contener las columnas **join_keys** especificadas en las **entities** asociadas con el **feature view**. En este ejemplo, estamos usando datos de series tempor



In [ ]:
managed_fv = FeatureView(
    name="Customer_Features",
    entities = [customer_entity],
    feature_df=cust_sdf,
    
)
registered_fv = FS1.register_feature_view(
    feature_view=managed_fv,
    version="V1"
)

In [ ]:
managed_fv = FeatureView(
    name="Trans_Features",
    entities = [transaction_entity],
    feature_df=trans_sdf,
    
    
)

registered_fv = FS1.register_feature_view(
    feature_view=managed_fv,
    version="V1"
)

Esto completa la configuración de los objetos de la base de datos y el flujo de trabajo del **Feature Store Producer**. Los datos y las **features** que se han generado están disponibles para el **consumer** con los privilegios apropiados. ¡Es hora de continuar al siguiente notebook!
